In [16]:
from gurobipy import *

In [17]:
def clean(row,i):
    b=row.strip().replace("\xc2\xa0", "").replace("\xef\xbb\xbf", "").split(',')[i]
    return b

In [18]:
customer=dict()

for i,row in enumerate(open('test2.csv')):
    if i >= 1:
        customer[i-1]=dict()
        customer[i-1]['X']=float(clean(row,1))
        customer[i-1]['Y']=float(clean(row,2))
        customer[i-1]['STIME']=float(clean(row,3))
        customer[i-1]['DEMAND']=float(clean(row,4))
        customer[i-1]['OPEN']=float(clean(row,5))
        customer[i-1]['DUE']=float(clean(row,6))
n = len(customer)
#新增end node
customer[n]=dict()
customer[n]['X']=customer[0]['X']
customer[n]['Y']=customer[0]['Y']
customer[n]['STIME']=customer[0]['STIME']
customer[n]['DEMAND']=customer[0]['DEMAND']
customer[n]['OPEN']=customer[0]['OPEN']
customer[n]['DUE']=customer[0]['DUE']
n = len(customer) #更新node數量

In [19]:
K = 3 #vehicle number
cn = (len(customer)-1)/2 #the number of customers
L = 480 #route duration limit
P = list(range(1,cn+1)) #pick up nodes
D = list(range(cn+1,2*cn+1)) #delivery nodes
V = list(range(0,2*cn+2)) #set of all node, 0: depot, 49: end node
cap = 6 #vehicle capacity
ridetime = 90 #maximum user ride time
M = 10000

In [20]:
distance_matrix = []
for i in range(n):
    distance_list = []
    for j in range(n):
        if i == j:
            distance_list.append(0)
        else:
            distance_list.append(((customer[i]['X']-customer[j]['X'])**2+(customer[i]['Y']-customer[j]['Y'])**2)**0.5)
    distance_matrix.append(distance_list)

In [22]:
m = Model('Dial_a_Ride')
x = {}
T = {}
Q = {}
for i in V:
    for j in V:
        for k in range(K):
            x[i,j,k]=m.addVar(vtype=GRB.BINARY, name='x_%s_%s_%s' % (i, j, k))
for i in V:
    for k in range(K):
        Q[i,k] = m.addVar(lb = 0, name='Q_%s_%s' % (i, k))
for i in V:
    for k in range(K):
        T[i,k] = m.addVar(lb = 0, name='T_%s_%s' % (i, k))
        
m.update()


m.setObjective(quicksum(distance_matrix[i][j]*x[i,j,k] for i in V for j in V for k in range(K)), GRB.MINIMIZE)

for i in P:
    m.addConstr(quicksum(x[i,j,k] for j in V for k in range(K))== 1 ,'service_once_%s' % (i))
for i,l in zip(P,D):
    for k in range(K):
        m.addConstr(quicksum(x[i,j,k] for j in V) - quicksum(x[l,j,k] for j in V) == 0, 'service_same_%s%s' % (i,k))
for k in range(K):
    m.addConstr(quicksum(x[0,j,k] for j in V) == 1, 'start_from_depot_%s' % (k))
for i in (P+D):
    for k in range(K):
        m.addConstr(quicksum(x[j,i,k] for j in V) - quicksum(x[i,l,k] for l in V) == 0, 'flowinout_%s%s' % (i,k))
for k in range(K):
    m.addConstr(quicksum(x[i,n-1,k] for i in V) == 1, 'back_to_depot_%s' % (k))

for k in range(K):
    m.addConstr(quicksum(x[j,0,k] for j in V) == 0, 'notin_depot_%s' % (k))

for k in range(K):
    m.addConstr(quicksum(x[n-1,i,k] for i in V) == 0, 'notout_depot_%s' % (k))        

for i in P:
    for k in range(K):
        m.addConstr(x[i,i,k]  == 0, 'not_same_node_%s' % (k))
'''
for i in V:
    for k in range(K):
        if i != n-1:
            m.addConstr(T[n-1,k] >= T[i,k]+distance_matrix[i][n-1],'con1')
'''
for i in V:
    for j in V:
        for k in range(K):
            m.addConstr(T[j,k] >= (T[i,k]+customer[i]['STIME']+distance_matrix[i][j])*x[i,j,k], 'time_%s%s%s' % (i,j,k))
for i in V:
    for j in V:
        for k in range(K):
            m.addConstr(Q[j,k] >= (Q[i,k]+customer[i]['DEMAND'])*x[i,j,k], 'load_%s%s%s' % (i,j,k))
for i,l in zip(P,D):
    for k in range(K):
        m.addConstr(T[l,k]-T[i,k]-customer[i]['STIME']-distance_matrix[i][l] >=0, 'sequence_%s' % (i))
for k in range(K):
    m.addConstr(T[n-1,k]-T[0,k] <= L, 'durationlimit_%s' % (k))
for i in V:
    for k in range(K):
        m.addConstr(T[i,k] <= customer[i]['DUE'], 'duetime_%s%s' % (i,k))
for i in V:
    for k in range(K):
        m.addConstr(T[i,k] >= customer[i]['OPEN'], 'starttime_%s%s' % (i,k))
for i in V:
    for k in range(K):
        m.addConstr(Q[i,k] <= min(cap, cap+customer[i]['DEMAND']), 'cap_upperbound_%s%s' % (i,k))
for i in V:
    for k in range(K):
        m.addConstr(Q[i,k] >= max(0, customer[i]['DEMAND']), 'cap_lowerbound_%s%s' % (i,k))
for i,l in zip(P,D):
    for k in range(K):
        m.addConstr(T[l,k] - T[i,k] <= ridetime ,'ridetime_limit_%s%s' % (i,k))

#m.setParam('OutputFlag', False)

m.setParam('TimeLimit', 3600)
#m.setParam('MIPGAP',1)

m.optimize()

Changed value of parameter MIPGAP to 1.0
   Prev: 0.0001  Min: 0.0  Max: 1e+100  Default: 0.0001
Optimize a model with 153 rows, 240 columns and 690 nonzeros
Model has 384 quadratic constraints
Variable types: 48 continuous, 192 integer (192 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  QMatrix range    [1e+00, 1e+00]
  QLMatrix range   [1e+00, 1e+04]
  Objective range  [2e+00, 1e+04]
  Bounds range     [1e+00, 1e+03]
  RHS range        [1e+00, 1e+03]
Presolve removed 102 rows and 80 columns
Presolve time: 0.06s
Presolved: 483 rows, 376 columns, 1667 nonzeros
Variable types: 219 continuous, 157 integer (112 binary)

Root relaxation: objective 1.628187e+01, 38 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   16.28187    0   12          -   16.28187      -     -    0s
H    0     0                      33.5104241   16.28187  5

In [6]:
'''
m = Model('Dial_a_Ride')
x = {}
T = {}
Q = {}
for i in V:
    for j in V:
        for k in range(K):
            x[i,j,k]=m.addVar(obj = distance_matrix[i][j], vtype=GRB.BINARY, name='x_%s_%s_%s' % (i, j, k))
for i in V:
    for k in range(K):
        Q[i,k] = m.addVar(name='Q_%s_%s' % (i, k))
for i in V:
    for k in range(K):
        T[i,k] = m.addVar(ub = 1440, name='T_%s_%s' % (i, k))
        
m.update()


m.setObjective(quicksum(distance_matrix[i][j]*x[i,j,k] for i in V for j in V for k in range(K)), GRB.MINIMIZE)

for i in P:
    m.addConstr(quicksum(x[i,j,k] for j in V for k in range(K))== 1 ,'service_once_%s' % (i))
for i,l in zip(P,D):
    for k in range(K):
        m.addConstr(quicksum(x[i,j,k] for j in V) - quicksum(x[l,j,k] for j in V) == 0, 'service_same_%s%s' % (i,k))
for k in range(K):
    m.addConstr(quicksum(x[0,j,k] for j in V) == 1, 'start_from_depot_%s' % (k))
for i in (P+D):
    for k in range(K):
        m.addConstr(quicksum(x[j,i,k] for j in V) - quicksum(x[i,l,k] for l in V) == 0, 'flowinout_%s%s' % (i,k))
for k in range(K):
    m.addConstr(quicksum(x[i,n-1,k] for i in V) == 1, 'back_to_depot_%s' % (k))
for k in range(K):
    m.addConstr(quicksum(x[j,0,k] for j in V) == 0, 'notin_depot_%s' % (k))
for k in range(K):
    m.addConstr(quicksum(x[n-1,i,k] for i in V) == 0, 'notout_depot_%s' % (k))        
    
for i in V:
    for j in V:
        for k in range(K):
            m.addConstr(T[j,k] >= (T[i,k]+customer[i]['STIME']+distance_matrix[i][j])*x[i,j,k], 'time_%s%s%s' % (i,j,k))
for i in V:
    for j in V:
        for k in range(K):
            m.addConstr(Q[j,k] >= (Q[i,k]+customer[i]['DEMAND'])*x[i,j,k], 'load_%s%s%s' % (i,j,k))
for i,l in zip(P,D):
    m.addConstr(T[l,k]-T[i,k]-customer[i]['STIME']-distance_matrix[i][l] >=0, 'sequence_%s' % (i))

for k in range(K):
    m.addConstr(T[n-1,k]-T[0,k] <= L, 'durationlimit_%s' % (k))
for i in V:
    for k in range(K):
        m.addConstr(T[i,k] <= customer[i]['DUE'], 'duetime_%s%s' % (i,k))
for i in V:
    for k in range(K):
        m.addConstr(T[i,k] >= customer[i]['OPEN'], 'starttime_%s%s' % (i,k))
for i in V:
    for k in range(K):
        mq = min(cap, cap+customer[i]['DEMAND'])
        m.addConstr(Q[i,k] <= mq, 'cap_upperbound_%s%s' % (i,k))
for i in V:
    for k in range(K):
        mq = max(0, customer[i]['DEMAND'])
        m.addConstr(Q[i,k] >= mq, 'cap_lowerbound_%s%s' % (i,k))
for i,l in zip(P,D):
    for k in range(K):
        m.addConstr(T[l,k] - T[i,k] <= ridetime ,'ridetime_limit_%s%s' % (i,k))

#m.setParam('OutputFlag', False)

m.setParam('TimeLimit', 3600)
#m.setParam('MIPGAP',1)

m.optimize()
'''

Academic license - for non-commercial use only
Changed value of parameter TimeLimit to 3600.0
   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100
Optimize a model with 153 rows, 240 columns and 690 nonzeros
Model has 384 quadratic constraints
Variable types: 48 continuous, 192 integer (192 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  QMatrix range    [1e+00, 1e+00]
  QLMatrix range   [1e+00, 1e+04]
  Objective range  [2e+00, 1e+04]
  Bounds range     [1e+00, 1e+03]
  RHS range        [1e+00, 1e+03]
Presolve removed 102 rows and 80 columns
Presolve time: 0.05s
Presolved: 483 rows, 376 columns, 1667 nonzeros
Variable types: 219 continuous, 157 integer (112 binary)

Root relaxation: objective 1.628187e+01, 38 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   16.28187    0   12          -   16.28187      -     -    0s
H    

In [23]:
print 'Obj: %f' % m.objVal

Obj: 33.510424


In [24]:
for v in m.getVars():
    if v.x > 0:
        print '%s: %f' % (v.varName, v.x)

x_0_3_2: 1.000000
x_0_7_0: 1.000000
x_0_7_1: 1.000000
x_1_4_2: 1.000000
x_2_5_2: 1.000000
x_3_6_2: 1.000000
x_4_2_2: 1.000000
x_5_7_2: 1.000000
x_6_1_2: 1.000000
Q_1_0: 1.000000
Q_1_1: 1.000000
Q_1_2: 1.000000
Q_2_0: 1.000000
Q_2_1: 1.000000
Q_2_2: 1.000000
Q_3_0: 1.000000
Q_3_1: 1.000000
Q_3_2: 1.000000
Q_4_2: 2.000000
Q_5_0: 1.000000
Q_5_1: 1.000000
Q_5_2: 2.000000
Q_6_2: 2.000000
Q_7_0: 6.000000
Q_7_1: 6.000000
Q_7_2: 6.000000
T_0_0: 141.592655
T_0_1: 141.592655
T_0_2: 112.624228
T_1_0: 168.000000
T_1_1: 168.000000
T_1_2: 228.600698
T_2_0: 239.000000
T_2_1: 239.000000
T_2_2: 270.569432
T_3_0: 147.968427
T_3_1: 147.968427
T_3_2: 147.968427
T_4_0: 258.000000
T_4_1: 258.000000
T_4_2: 258.000000
T_5_0: 329.000000
T_5_1: 329.000000
T_5_2: 329.000000
T_6_0: 209.000000
T_6_1: 209.000000
T_6_2: 209.000000
T_7_0: 621.592655
T_7_1: 621.592655
T_7_2: 592.624228


In [15]:
cn

24